In [1]:
import os
os.environ["KERAS_BACKEND"] = "jax"
import jax
import keras

In [5]:
import keras.layers

class CustomerTicketModel(keras.Model):
    def __init__(self, num_dept):
        super().__init__()
        self.concat_layer = keras.layers.Concatenate()
        self.mixing_layer = keras.layers.Dense(64, activation="relu")
        self.priority_scorer = keras.layers.Dense(1, activation="sigmoid")
        self.department_classifier = keras.layers.Dense(num_dept, activation="softmax")

    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]
        cct = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(cct)
        priority = self.priority_scorer(features)
        dept = self.department_classifier(features)
        return priority, dept
        
        

In [9]:
import numpy as np

n = 1280
vocab_size = 10000
num_tags = 100
num_departments = 4
title_data = np.random.randint(0, 2, size=(n, vocab_size))
text_body_data = np.random.randint(0, 2, size=(n, vocab_size))
tags_data = np.random.randint(0,2, size=(n, num_tags))
priority_data = np.random.random(size=(n, 1))
department_data = np.random.randint(0, 2, size=(n, num_departments))
model = CustomerTicketModel(num_departments)

In [11]:
input_dict = {
    "title": title_data, "text_body": text_body_data,
    "tags": tags_data }
priority, department = model(input_dict)

In [12]:
model.compile(optimizer="rmsprop", loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"],["accuracy"]])
model.fit(input_dict, [priority_data, department_data], epochs=1)

/Users/ethanvizitei/Code/Personal/dlwp/venv/lib/python3.10/site-packages/jax/_src/interpreters/mlir.py:1114: UserWarning: Some donated buffers were not usable: ShapedArray(float32[20100,64]), ShapedArray(float32[64]), ShapedArray(float32[64,1]), ShapedArray(float32[1]), ShapedArray(float32[64,4]), ShapedArray(float32[4]), ShapedArray(int32[]), ShapedArray(float32[]), ShapedArray(float32[20100,64]), ShapedArray(float32[64]), ShapedArray(float32[64,1]), ShapedArray(float32[1]), ShapedArray(float32[64,4]), ShapedArray(float32[4]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]).
Donation is not implemented for ('METAL',).
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn("Some donated buffers were not usable:"


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4877 - categorical_crossentropy_loss: 21.9440 - loss: 22.2484 - mean_absolute_error: 0.4719 - mean_squared_error_loss: 0.3044


/Users/ethanvizitei/Code/Personal/dlwp/venv/lib/python3.10/site-packages/jax/_src/interpreters/mlir.py:1114: UserWarning: Some donated buffers were not usable: ShapedArray(float32[20100,64]), ShapedArray(float32[64]), ShapedArray(float32[64,1]), ShapedArray(float32[1]), ShapedArray(float32[64,4]), ShapedArray(float32[4]), ShapedArray(int32[]), ShapedArray(float32[]), ShapedArray(float32[20100,64]), ShapedArray(float32[64]), ShapedArray(float32[64,1]), ShapedArray(float32[1]), ShapedArray(float32[64,4]), ShapedArray(float32[4]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]).
Donation is not implemented for ('METAL',).
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn("Some donated buffers were not usable:"


In [14]:
model.evaluate(input_dict, [priority_data, department_data])

/Users/ethanvizitei/Code/Personal/dlwp/venv/lib/python3.10/site-packages/jax/_src/interpreters/mlir.py:1114: UserWarning: Some donated buffers were not usable: ShapedArray(float32[20100,64]), ShapedArray(float32[64]), ShapedArray(float32[64,1]), ShapedArray(float32[1]), ShapedArray(float32[64,4]), ShapedArray(float32[4]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]).
Donation is not implemented for ('METAL',).
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn("Some donated buffers were not usable:"


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5482 - categorical_crossentropy_loss: 24.3343 - loss: 24.6788 - mean_absolute_error: 0.5074 - mean_squared_error_loss: 0.3445


[24.3387508392334,
 0.5045356750488281,
 0.5726562738418579,
 0.33789998292922974,
 24.000844955444336]